# Part 2: Logistic Regression and Gradient Checking

In [ ]:
# Execute this code block to install dependencies when running on colab
try:
    import torch
except:
    from os.path import exists
    from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
    platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
    cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
    accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

    !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-1.0.0-{platform}-linux_x86_64.whl torchvision

In the first part of the lab we saw how to make predictions of continously varying values with a linear regression model. Lets now turn our focus to binary classification using a simple classification algorithm known as Logistic regression.

In linear regression we tried to predict the value of $y$ for an example $\mathbf{x}$ using a linear function $y=\mathbf{x}^\top\theta$ (where $\mathbf{x}$ and $\theta$ are column-vectors). This will clearly not be a great solution for predicting binary-valued labels ($y\in\{0,1\}$). In logistic regression we use a different hypothesis class to try to predict the probability that a given example belongs to the "1" class versus the probability that it belongs to the "0" class. Specifically, we will try to learn a function of the form:

\begin{align}
P(y=1|\mathbf{x}) &= \frac{1}{1 + \exp(-\mathbf{x}^\top\theta)} \equiv \sigma(\mathbf{x}^\top\theta),\\
P(y=0|\mathbf{x}) &= 1 - P(y=1|\mathbf{x}).
\end{align}
 
The function $\sigma(z) \equiv \frac{1}{1 + \exp(-z)}$ is called the "sigmoid" or "logistic" function. The sigmoid function squashes any real valued input into the range $[0,1]$ enabling us to interprete the output as a probability. Our goal is to search for a value of $\theta$ so that the probability $P(y=1|\mathbf{x})=\sigma(\mathbf{x}^\top\theta)$ is large when $\mathbf{x}$ belongs to the "1" class and small when $\mathbf{x}$ belongs to the "0" class (so that $P(y=0|\mathbf{x})$ is large). 

With Linear Regression, the natural cost function was one that measured the sum of squared residuals (the difference between the predicted value and true value). With logisitic regression we have a probabilisitic model, so it makes sense that we use a function that measures the likelihood of the data given the model (note that we want to maximise this function rather than minimise it). As an aside, note that in the case of linear regression if we assume that the data has errors that are IID (independently and identically distributed) according to a Normal distribution, then it can be shown that the maximising the likelihood is exactly the same as minimising the sum of squared residuals. For logistic regression, the likelihood function for a single data point is:

\begin{align}
p(y|\mathbf{x}; \theta) &= \sigma(\mathbf{x}^\top\theta)^y(1-\sigma(\mathbf{x}^\top\theta)^{(1-y)}.
\end{align}

For a complete dataset of points $(y_i, \mathbf{x}_i)$, then the complete likelihood is:

\begin{align}
L(\theta) &= \prod_i \sigma(\mathbf{x}_i^\top\theta)^{y_i}(1-\sigma(\mathbf{x}_i^\top\theta)^{(1-y_i)}
\end{align}

However, it is considerably easier to maximise the log-likelihood function:

\begin{align}
\mathcal{l}(\theta) &= \log L(\theta) \\
                    & = \log \prod_i \sigma(\mathbf{x}_i^\top\theta)^{y_i}(1-\sigma(\mathbf{x}_i^\top\theta)^{(1-y_i)} \\
                    & = \sum_i y_i \log(\sigma(\mathbf{x}_i^\top\theta)) + (1-y_i) \log(1-\sigma(\mathbf{x}_i^\top\theta))
\end{align}

Clearly, maximising the log-likelihood is equivalent to minimising the negative log-likelihood. The negative of the log-likelihood function having the form $-\sum_i y_i \log(p) + (1-y_i) \log(p)$, where p is a function returning the predicted probability of class "1", is often called the __"Binary Cross Entropy"__ function, __"Binary Cross Entropy Loss"__ or sometimes the __"log loss"__.

For conciseness and computational efficiency, we can write the negative logistic regression log-likelihood function in matrix form. Assuming the $y_i$ are stored in a column vector $\mathbf{y}$ and the data vectors $x_i$ in the __rows__ of a matrix $\mathbf{X}$, then: 

\begin{align}
\mathrm{NLL}(\theta) & = -(\mathbf{y} \log(\sigma(\mathbf{X}\theta)) + (1-\mathbf{y}) \log(1-\sigma(\mathbf{X}\theta)))
\end{align}

The gradients of this function are given by:

\begin{align}
\nabla_\theta \mathrm{NLL}(\theta) & = \mathbf{X}^T(\sigma(\mathbf{X}\theta) - \mathbf{y})
\end{align}


__Use the box below to compute the gradients of the negative log-likelihood function $\nabla_\theta \mathrm{NLL}(\theta)$. You can use `torch.sigmoid()` to apply the sigmoid function.__

In [ ]:
import torch

def logistic_regression_loss_grad(theta, X, y):
    # YOUR CODE HERE
    raise NotImplementedError()
    return grad

In [ ]:
from sklearn.datasets import load_digits

X, y = tuple(torch.Tensor(z) for z in load_digits(2, True)) #convert to pytorch Tensors
X = torch.cat((X, torch.ones((X.shape[0], 1))), 1) # append a column of 1's to the X's
y = y.reshape(-1, 1) # reshape y into a column vector

print(y.shape)
# We're also going to break the data into a training set for computing the regression parameters
# and a test set to evaluate the predictive ability of those parameters
perm = torch.randperm(y.shape[0])
X_train = X[perm[0:260], :]
y_train = y[perm[0:260]]
X_test = X[perm[260:], :]
y_test = y[perm[260:]]

In [ ]:
alpha = 0.001
theta_gd = torch.rand((X_train.shape[1], 1))
for e in range(0, 10):
    gr = logistic_regression_loss_grad(theta_gd, X_train, y_train)
    theta_gd -= alpha * gr

print("Gradient Descent Theta: ", theta_gd.t())
print("BCE of test data: ", torch.nn.functional.binary_cross_entropy_with_logits(X_test @ theta_gd, y_test))